In [ ]:
import numpy as np
import h5py
import tomopy
import matplotlib.pyplot as plt

# h5 Datei laden
with h5py.File('phase_recos.h5', 'r') as file:
    projections = file['phase_recos'][:]

# Geometrie aus tomoreco.py übernommen
rotation_center = 128
rotation_axis = 0
rotation_angles = np.linspace(0, 180, projections.shape[0])
z01 = 11.8e-3
z02 = 221.5e-3
M = 18.770049825950448
px = 55e-6

# Pixelgrößre berechnet
effective_pixel_size = px * M / z02

# Flat field Region bestimmt, Kugelinneres verwendet
uniform_region = projections[:, 400:500, 350:450]
flat_field = np.mean(uniform_region)

# Korrektion durchgeführt
projections_corrected = projections / flat_field

# Bad Pixel Korrektion
threshold = 0.8  # Arbiträre Hürde gewählt
bad_pixels = np.where(projections_corrected < threshold)

projections_corrected[bad_pixels] = 0

# Filter erstellt
reconstruction_algorithm = 'gridrec'
reconstruction_options = {'filter_name': 'parzen', 'filter_par': 0.5, 'ncore': 1}

# Rekonstruktion durchgeführt
reconstruction = tomopy.recon(projections_corrected, rotation_angles, algorithm=reconstruction_algorithm, **reconstruction_options)

# .h5 Datei erstellt
with h5py.File('reconstructed_image2.h5', 'w') as file:
    file.create_dataset('reconstructed_image', data=reconstruction)

# Ein Bildr wird dargestellt
plt.imshow(reconstruction[0], cmap='gray')
plt.title('Reconstructed Image')
plt.xlabel('Pixel')
plt.ylabel('Pixel')
plt.colorbar(label='Intensity')

In [ ]:
import numpy as np
import h5py
import tomopy
import matplotlib.pyplot as plt
import imageio
import os

# h5 Datei laden
with h5py.File('phase_recos.h5') as f:
    projections = f['phase_recos'][:]

# Parameter
rotation_axis = 0
rotation_angles_deg = np.linspace(0, 180, projections.shape[0])
rotation_angles_rad = np.deg2rad(rotation_angles_deg)

# Rotationscenter berechen (335)
rot_center = tomopy.find_center(projections, rotation_angles_rad, init=290, ind=0, tol=0.5)

# Algo
reconstruction_algorithm = 'gridrec'
reconstruction_options = {}

# Rekonstruktion
reconstruction = tomopy.recon(projections, rotation_angles_rad, algorithm=reconstruction_algorithm, center=rot_center, sinogram_order=False, **reconstruction_options)

# Create a directory to save the images
output_dir = 'reconstruction_images'
os.makedirs(output_dir, exist_ok=True)

# Save each slice as an image
for i in range(reconstruction.shape[0]):
    plt.imshow(reconstruction[i], cmap='gray')
    plt.axis('off')
    plt.savefig(f'{output_dir}/reconstruction_{i:03d}.png')
    plt.close()

# Create a GIF from the saved images
images = []
for i in range(reconstruction.shape[0]):
    filename = f'{output_dir}/reconstruction_{i:03d}.png'
    images.append(imageio.imread(filename))
imageio.mimsave('reconstruction.gif', images)

# Display the reconstructed volume
plt.imshow(reconstruction[0], cmap='gray')
plt.show()